In [1]:
import numpy as np
import torch
import torch.nn as tnn
import torch.nn.functional as F
import torch.optim as topti
from torchtext import data
from torchtext.vocab import GloVe
from imdb_dataloader import IMDB

# 构建神经网络

In [18]:
# Class for creating the neural network.
class Network(tnn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.conv1 = torch.nn.Conv1d(50, 50, 8, padding=5,)
        self.relu = torch.nn.ReLU()
        self.pool1 = torch.nn.MaxPool1d(4)
        self.norm = torch.nn.BatchNorm2d(50)
        self.conv2 = torch.nn.Conv1d(50, 50, 8, padding=5)
        self.dropout = torch.nn.Dropout()
        # ReLu
        self.pool2 = torch.nn.MaxPool1d(4) # shape (batch_size, channel)
        self.conv3 = torch.nn.Conv1d(50, 50, 8, padding=5)
        # ReLu
        self.global_pool = torch.nn.functional.max_pool1d
        self.dense = torch.nn.Linear(50, 1)

    def forward(self, input, length):
        """
        DO NOT MODIFY FUNCTION SIGNATURE
        Create the forward pass through the network.
        """
        x = self.conv1(input.permute(0,2,1))
        x = self.relu(x)
        x = self.pool1(x)
        x = self.norm(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool2(x)
        x = self.conv3(x)
        x = self.relu(x)
        x = self.global_pool(x,kernel_size=x.shape[2]).view(-1,50)
        x = self.dropout(x)
        x = self.dense(x)
        return (x)

        # TODO Weight Normalization
        return x

In [ ]:
# 83.01856594110116
# 83.67477592829705
# 83.91485275288092

In [19]:
class PreProcessing():

    def pre(x):
        
        """Called after tokenization 把句子拆成单词 """
        """
        GRAM too small ....
        """
#         _arr = np.array([item for sublist in x for item in sublist]).reshape(1,-1)
#         for i,dim in enumerate(_arr):
#             _index = np.argwhere((dim == '/><br') == True).flatten()
#             for j in _index:
#                 _arr[i][j] = ""|
#                 try:
#                     _arr[i][j-1] =_arr[i][j-1].replace("<br")
#                     _arr[i][j+1] =_arr[i][j+1].replace("/>")
#                 except IndexError:
#                     print("outofbound")
#         #return _arr.flatten().tolist()
        return x
    def post(batch, vocab):
        
        
        #return torch.nn.functional.normalize(torch.FloatTensor(batch))
        return batch

    text_field = data.Field(lower=True, include_lengths=True, batch_first=True, preprocessing=pre, postprocessing=post)

In [20]:
def lossFunc():
    """
    Define a loss function appropriate for the above networks that will
    add a sigmoid to the output and calculate the binary cross-entropy.
    """
    return tnn.BCEWithLogitsLoss()

In [21]:
def main():
    # Use a GPU if available, as it should be faster.
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("Using device: " + str(device))
    torch.cuda.empty_cache()
    # Load the training dataset, and create a data loader to generate a batch.
    textField = PreProcessing.text_field
    labelField = data.Field(sequential=False)

    train, dev = IMDB.splits(textField, labelField, train="train", validation="dev")

    textField.build_vocab(train, dev, vectors=GloVe(name="6B", dim=50))
    labelField.build_vocab(train, dev)

    trainLoader, testLoader = data.BucketIterator.splits((train, dev), shuffle=True, batch_size=64,
                                                         sort_key=lambda x: len(x.text), sort_within_batch=True)

    net = Network().to(device)
    
    criterion =lossFunc()
    optimiser = topti.Adam(net.parameters(), lr=1e-3)  # Minimise the loss using the Adam algorithm.
    scheduler = torch.optim.lr_scheduler.StepLR(optimiser,step_size=2, gamma=0.7)
    for epoch in range(10):
        running_loss = 0
        for i, batch in enumerate(trainLoader):
            # Get a batch and potentially send it to GPU memory.
            inputs, length, labels = textField.vocab.vectors[batch.text[0]].to(device), batch.text[1].to(
                device), batch.label.type(torch.FloatTensor).to(device)

            labels -= 1

            # PyTorch calculates gradients by accumulating contributions to them (useful for
            # RNNs).  Hence we must manually set them to zero before calculating them.
            optimiser.zero_grad()
            
            # Forward pass through the network.
            output = net(inputs, length)

            loss = criterion(output.view(-1), labels)

            # Calculate gradients.
            loss.backward()

            # Minimise the loss according to the gradient.
            optimiser.step()
            
            running_loss += loss.item()
        
            if i % 32 == 31:
                print("Epoch: %2d, Batch: %4d, Loss: %.3f" % (epoch + 1, i + 1, running_loss / 32))
                running_loss = 0
        scheduler.step()
    num_correct = 0

    # Save mode



    # Evaluate network on the test dataset.  We aren't calculating gradients, so disable autograd to speed up
    # computations and reduce memory usage.
    with torch.no_grad():
        for batch in testLoader:
            # Get a batch and potentially send it to GPU memory.
            inputs, length, labels = textField.vocab.vectors[batch.text[0]].to(device), batch.text[1].to(
                device), batch.label.type(torch.FloatTensor).to(device)

            labels -= 1

            # Get predictions
            outputs = torch.sigmoid(net(inputs, length))
            predicted = torch.round(outputs.view(-1))
            num_correct += torch.sum(labels == predicted).item()

    accuracy = 100 * num_correct / len(dev)
    print(f"Classification accuracy: {accuracy}")


if __name__ == '__main__':
    main()

Using device: cuda:0
Epoch:  1, Batch:   32, Loss: 0.695
Epoch:  1, Batch:   64, Loss: 0.684
Epoch:  1, Batch:   96, Loss: 0.654
Epoch:  1, Batch:  128, Loss: 0.591
Epoch:  1, Batch:  160, Loss: 0.543
Epoch:  1, Batch:  192, Loss: 0.518
Epoch:  1, Batch:  224, Loss: 0.513
Epoch:  1, Batch:  256, Loss: 0.479
Epoch:  1, Batch:  288, Loss: 0.518
Epoch:  1, Batch:  320, Loss: 0.487
Epoch:  1, Batch:  352, Loss: 0.481
Epoch:  1, Batch:  384, Loss: 0.489
Epoch:  2, Batch:   32, Loss: 0.423
Epoch:  2, Batch:   64, Loss: 0.443
Epoch:  2, Batch:   96, Loss: 0.439
Epoch:  2, Batch:  128, Loss: 0.443
Epoch:  2, Batch:  160, Loss: 0.429
Epoch:  2, Batch:  192, Loss: 0.437
Epoch:  2, Batch:  224, Loss: 0.406
Epoch:  2, Batch:  256, Loss: 0.398
Epoch:  2, Batch:  288, Loss: 0.406
Epoch:  2, Batch:  320, Loss: 0.425
Epoch:  2, Batch:  352, Loss: 0.447
Epoch:  2, Batch:  384, Loss: 0.437
Epoch:  3, Batch:   32, Loss: 0.387
Epoch:  3, Batch:   64, Loss: 0.380
Epoch:  3, Batch:   96, Loss: 0.362
Epoch: 

In [ ]:
82.93854033290653